# Battle of the Neighborhoods: Capstone Project


**Coursera IBM Data Science Capstone Project**   
May 2020  
by Claire Li

# Table of Contents
1. [Introduction/Business Problem](#introduction)
2. [Methodology Overview](#overview)
3. [Part 1: Demographic Data](#demodata)  
    1. [Preparing the Data](#dataLA)   
    2. [Clustering](#clusterLA) 
4. [Part 2: Foursquare Data](#foursquare)
    1. [Preparing the Data](#datafoursquare)   
    2. [Clustering](#clusterfoursquare) 
5. [Results](#Results)

## 1. Introduction/Business Problem <a name="introduction"></a>
   The client is the owner of a fast-casual Chinese restaurant in Irvine, California, which is located in the suburb of Orange County. As the restaurant has proven to be a success in its years of business, the client is seeking to take the next step and expand to the Los Angeles metropolitan area. 
   
As a bustling urban area with some of the nation’s highest rent costs, it is a considerable investment to open a storefront in Los Angeles, but also promises the opportunity for high returns and rapid growth. The client is seeking a location for their new venture that shares similar characteristics with its existing location in Orange County. Specifically, they are looking for a similar demographic profile as well as a similar competitor landscape. Therefore, the aim of this study is to research and suggest neighborhoods for the client’s proposed new venture. Specifically, the business problem is: **what neighborhoods in Los Angeles have a similar demographic profile and competitor landscape to Irvine, California, and are potential locations for the client’s proposed new restaurant?** 




## 2. Methodology Overview <a name="overview"></a>
The features in particular that we seek in an potential location include:
* A similar demographic profile to the original Irvine location. Factors to be included in the analysis are: age distribution, median income, and race/ethnicity. 
* Similar consumer preferences. This will be measured by the types of businesses in the area.  
* A favorable competitor landscape. Ideally, we seek an area with fewer Chinese restaurants already in place. 

This study was conducted in two parts. First, we examine the demographic composition of various LA neighborhoods in order to find a similar demographic area to the original Irvine location. Second, we study competitor landscape and consumer preferences via the Foursquare API. 

### Los Angeles Neighborhood Data
We will use the list of L.A. County neighborhoods as defined by the Los Angeles Times, available here: 
http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-v6.geojson

### Los Angeles Demographic Statistics
We are interested in identifying a neighborhood of Los Angeles with comparable characteristics to the original restaurant location in Orange County. To do this, we will use data about age distribution (available here: https://usc.data.socrata.com/Los-Angeles/Age-Distribution-LA-/rqg9-k6ju/data), income (available here: https://usc.data.socrata.com/Los-Angeles/Income-LA-/kygc-fzgm/data [**Edit**: this is now unavailable; this one was used instead: https://maps.latimes.com/neighborhoods/income/median/neighborhood/list/]), and race/ethnicity (available here: https://usc.data.socrata.com/Los-Angeles/Race-Ethnicity-LA-/jxw5-xxv5/data). 

### Irvine Demographic Statistics
We are specifically interested in the demographic data of Irvine, California, as provided by the US Census Bureau: https://www.census.gov/quickfacts/fact/table/irvinecitycalifornia/PST045219? and
https://data.census.gov/cedsci/table?q=%20irvine&g=1600000US0636770&tid=ACSDP1Y2018.DP05&hidePreview=true

### Foursquare API 
Foursquare is a local search-and-discovery platform. We will utilize the Foursquare API for location-based insights on local venues in LA neighborhoods in order to 1.) Find a neighborhood with fewer extant Chinese restaurants and 2.) Determine the top types of venues for each neighborhood, to get a sense for consumer preferences. 


## 3. Part 1: Demographic Data  <a name="demodata"></a>

In this section, we analyze the demographic information for LA and Irvine. 

### Preparing the Data <a name="dataLA"></a>   
We will first work with the Los Angeles data. The first step is to get a list of all the Los Angeles neighborhoods, along with their coordinates. We use data made available by the LA Times. 

#### Los Angeles Neighborhood Data 

In [6]:
#Import libraries
import numpy as np
import pandas as pd
! pip install geopandas
import geopandas as gpd 
! pip install --user folium
import folium
import json 
import requests 

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.colors as colors

print("Libraries imported")

     |████████████████████████████████| 102kB 18.3MB/s ta 0:00:01
Libraries imported


In [3]:
#Download the geojson file from the LATimes
!wget -O LA_neighborhoods.geojson http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-current.geojson
print("DONE")

#Use geopandas to read the geojson file 
la_geo = r"LA_neighborhoods.geojson" #geojson file
neighborhoods_data = gpd.read_file(la_geo)
neighborhoods_data.head()

--2020-05-13 22:05:47--  http://s3-us-west-2.amazonaws.com/boundaries.latimes.com/archive/1.0/boundary-set/la-county-neighborhoods-current.geojson
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.193.120
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.193.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1345788 (1.3M) [application/geo+json]
Saving to: ‘LA_neighborhoods.geojson’

100%[======================================>] 1,345,788   3.62MB/s   in 0.4s   

2020-05-13 22:05:48 (3.62 MB/s) - ‘LA_neighborhoods.geojson’ saved [1345788/1345788]

DONE


,kind,external_id,name,slug,set,metadata,resource_uri,geometry
0,L.A. County Neighborhood (Current),acton,Acton,acton-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 39.3391089485, 'type': 'unincorporate...",/1.0/boundary/acton-la-county-neighborhood-cur...,"MULTIPOLYGON (((-118.20262 34.53899, -118.1894..."
1,L.A. County Neighborhood (Current),adams-normandie,Adams-Normandie,adams-normandie-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 0.805350187789, 'type': 'segment-of-a...",/1.0/boundary/adams-normandie-la-county-neighb...,"MULTIPOLYGON (((-118.30901 34.03741, -118.3004..."
2,L.A. County Neighborhood (Current),agoura-hills,Agoura Hills,agoura-hills-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 8.14676029818, 'type': 'standalone-ci...",/1.0/boundary/agoura-hills-la-county-neighborh...,"MULTIPOLYGON (((-118.76193 34.16820, -118.7263..."
3,L.A. County Neighborhood (Current),agua-dulce,Agua Dulce,agua-dulce-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 31.4626319451, 'type': 'unincorporate...",/1.0/boundary/agua-dulce-la-county-neighborhoo...,"MULTIPOLYGON (((-118.25468 34.55830, -118.2555..."
4,L.A. County Neighborhood (Current),alhambra,Alhambra,alhambra-la-county-neighborhood-current,/1.0/boundary-set/la-county-neighborhoods-curr...,"{'sqmi': 7.62381430605, 'type': 'standalone-ci...",/1.0/boundary/alhambra-la-county-neighborhood-...,"MULTIPOLYGON (((-118.12175 34.10504, -118.1168..."


In [7]:
#We will only work with the columns 'name', 'metadata', and 'geometry'
neighborhoods = neighborhoods_data[['name','metadata','geometry']]

#Add Longitude/Latitude coordinates using the centroid of the geographical region
neighborhoods['Longitude'] = neighborhoods.centroid.x
neighborhoods['Latitude'] = neighborhoods.centroid.y

#Drop unnecessary columns and rename columns 
neighborhoods.drop(columns = ['geometry', 'metadata'], axis = 1, inplace = True)
neighborhoods.rename(columns={'name':'Neighborhood'}, inplace=True)

neighborhoods.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors

,Neighborhood,Longitude,Latitude
0,Acton,-118.185799,34.495516
1,Adams-Normandie,-118.300288,34.031411
2,Agoura Hills,-118.760944,34.150734
3,Agua Dulce,-118.313371,34.508909
4,Alhambra,-118.135494,34.083967


We now have a list of the neighborhoods of LA along with their regions and coordinates. Let's plot this on a map to get a better visual. 

In [8]:
lat = 34.0522
long = -118.243683

# create map of LA using latitude and longitude values
map_la = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map
for lat, lng, name in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

#### Los Angeles Income Data 

Next, we examine some of the demographic information for each of the neighborhoods, starting with income data. This gives us a list of neighborhoods alongside their median income, which we then join with our existing LA neighborhoods dataframe.  

(**UPDATE**: The dataset I was using has mysteriously disappeared, so we are now using this less-thorough one.) 
https://maps.latimes.com/neighborhoods/income/median/neighborhood/list/


In [9]:
#Read the income data
la_income = pd.read_html('https://maps.latimes.com/neighborhoods/income/median/neighborhood/list/')[1]
la_income.drop(columns = ['Rank'], inplace = True)
la_income.head()

,Neighborhood,Median Income
0,Bel-Air,"$207,938"
1,Hidden Hills,"$203,199"
2,Rolling Hills,"$184,777"
3,Beverly Crest,"$169,282"
4,Pacific Palisades,"$168,008"


In [10]:
#Merge the income data with neighborhoods 
la_data = neighborhoods.merge(la_income, how = 'left', left_on = 'Neighborhood', right_on = 'Neighborhood')

#Drop all of the NaN values and cast income values as integers
la_data.dropna(inplace=True)
la_data['Median Income']= la_data['Median Income'].str.replace("$", "")
la_data['Median Income']= la_data['Median Income'].str.replace(",", "")
la_data['Median Income'] = pd.to_numeric(la_data['Median Income'], downcast = 'integer') 
la_data.head()

,Neighborhood,Longitude,Latitude,Median Income
0,Acton,-118.185799,34.495516,83983
1,Adams-Normandie,-118.300288,34.031411,29606
2,Agoura Hills,-118.760944,34.150734,117608
3,Agua Dulce,-118.313371,34.508909,106078
4,Alhambra,-118.135494,34.083967,53224


In [11]:
# The code was removed by Watson Studio for sharing.

#### Los Angeles Age Distribution Data 

Returning to our data, the next piece of information that we'd like to know is population and age distribution. 

In [12]:
#Get the data 
url = 'https://usc.data.socrata.com/resource/rqg9-k6ju.json?year=2018&$limit=20000'
response = requests.get(url)
data = response.json()
data
la_age_all = pd.DataFrame.from_records(data)

print(la_age_all.shape)
la_age_all.head()

(16408, 15)


,count,dataset,date,denominator,denominator_description,geoid,location,neighborhood,percent,policy_area,row_id,tract,tract_number,variable,year
0,1715,Age Distribution,2018-01-01T00:00:00.000,5705,Total Population,1400000US06037910713,"{'latitude': '34.57270418', 'longitude': '-118...",Palmdale,30.06135,Demography,Population_Under_Age_18_2018_1400000US06037910713,Census Tract 9107.13,910713,Population Under Age 18,2018
1,1206,Age Distribution,2018-01-01T00:00:00.000,4465,Total Population,1400000US06037920016,"{'latitude': '34.46388993', 'longitude': '-118...",Castaic Canyons,27.010078,Demography,Population_Under_Age_18_2018_1400000US06037920016,Census Tract 9200.16,920016,Population Under Age 18,2018
2,791,Age Distribution,2018-01-01T00:00:00.000,3170,Total Population,1400000US06037920036,"{'latitude': '34.42154774', 'longitude': '-118...",Santa Clarita,24.952681,Demography,Population_Under_Age_18_2018_1400000US06037920036,Census Tract 9200.36,920036,Population Under Age 18,2018
3,1591,Age Distribution,2018-01-01T00:00:00.000,4439,Total Population,1400000US06037910502,"{'latitude': '34.58002984', 'longitude': '-118...",Palmdale,35.841404,Demography,Population_Under_Age_18_2018_1400000US06037910502,Census Tract 9105.02,910502,Population Under Age 18,2018
4,NaN,Age Distribution,2018-01-01T00:00:00.000,NaN,Total Population,1400000US06037980025,"{'latitude': '33.84299739', 'longitude': '-118...",Carson,NaN,Demography,Population_Under_Age_18_2018_1400000US06037980025,Census Tract 9800.25,980025,Population Under Age 18,2018


Next, we clean the data so that we are only working with the relevant columns about age distribution.

In [13]:
#Drop all NaN values 
la_age_all.dropna(inplace=True)

#Only want the neighborhood, variable (= age category), count (= number in each age category), and denominator (= total population)
la_age = la_age_all[['neighborhood', 'variable', 'count', 'denominator']]
la_age.sort_values(by = 'neighborhood', inplace = True)

#Cast as int
la_age = la_age.astype({"count": int, "denominator": int})

#Sum up the total for each age group by neighborhood (neighborhoods potentially have mutliple tracts; we combine these to get a total for each neighborhood)
la_age = la_age.groupby(['neighborhood', 'variable']).agg({'count': 'sum', 'denominator': 'sum'})

#Calculate the percentage in each age group
la_age['Percentage'] = la_age['count']/la_age['denominator']
la_age.reset_index(drop = False, inplace = True)

#Rename columns and format percentage
la_age.rename(columns={'neighborhood':'Neighborhood', 'variable': 'Age Group', 'count': 'Count', 'denominator': 'Total Population'}, inplace=True)
la_age['Percentage'] = la_age['Percentage'] * 100

la_age.head(15)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Age Group,Count,Total Population,Percentage
0,Acton,Population Ages 18-24,635,7613,8.340996
1,Acton,Population Ages 25-34,736,7613,9.667674
2,Acton,Population Ages 35-44,782,7613,10.271903
3,Acton,Population Ages 45-54,1339,7613,17.588336
4,Acton,Population Ages 55-64,1568,7613,20.596348
5,Acton,Population Ages 65 & Older,1218,7613,15.998949
6,Acton,Population Under Age 18,1335,7613,17.535794
7,Adams-Normandie,Population Ages 18-24,3337,18237,18.297966
8,Adams-Normandie,Population Ages 25-34,2923,18237,16.027855
9,Adams-Normandie,Population Ages 35-44,2244,18237,12.304655


We then create a new DataFrame to display the percentage of each age group for each neighborhood. Then, we merge with the rest of our LA data.

In [14]:
#Create DataFrame displays age distribution and total population for each neighborhood 
percentages_list = la_age.groupby('Neighborhood')
result = percentages_list['Percentage'].unique()

percentages = pd.DataFrame(result)
percentages[['Ages 18-24','Ages 25-34', 'Ages 35-44','Ages 45-54','Ages 55-64', 'Ages 65 & Older', 'Under Age 18' ]] = pd.DataFrame(percentages.Percentage.tolist(), index= percentages.index)
percentages.drop(columns = ['Percentage'], inplace = True)
percentages = percentages[['Under Age 18', 'Ages 18-24','Ages 25-34', 'Ages 35-44','Ages 45-54','Ages 55-64', 'Ages 65 & Older']]

total_pop_list = la_age.groupby('Neighborhood')
result = total_pop_list['Total Population'].unique()
total_pop = pd.DataFrame(result)
total_pop = total_pop.astype({"Total Population": int})

la_age_final = percentages.merge(total_pop, how = 'outer', left_on = 'Neighborhood', right_on = 'Neighborhood')
la_age_final

#Merge with la_data 
la_data = la_data.merge(la_age_final, how = 'left', left_on = 'Neighborhood', right_on = 'Neighborhood')
print(la_data.shape)
la_data.head(40)

(264, 12)


,Neighborhood,Longitude,Latitude,Median Income,Under Age 18,Ages 18-24,Ages 25-34,Ages 35-44,Ages 45-54,Ages 55-64,Ages 65 & Older,Total Population
0,Acton,-118.185799,34.495516,83983,17.535794,8.340996,9.667674,10.271903,17.588336,20.596348,15.998949,7613.0
1,Adams-Normandie,-118.300288,34.031411,29606,20.946428,18.297966,16.027855,12.304655,11.443768,11.860503,9.118824,18237.0
2,Agoura Hills,-118.760944,34.150734,117608,22.520192,6.799345,9.771420,13.350578,17.763818,15.382991,14.411656,18943.0
3,Agua Dulce,-118.313371,34.508909,106078,17.722140,8.492882,12.714777,7.756505,15.635739,20.250368,17.427590,4074.0
4,Alhambra,-118.135494,34.083967,53224,17.292348,8.549674,16.339115,13.664180,14.331443,12.968673,16.854567,84974.0
5,Alondra Park,-118.334949,33.888520,57177,18.681319,9.442409,11.294261,10.826211,13.492063,15.934066,20.329670,4914.0
6,Altadena,-118.135550,34.193466,82676,19.430490,7.353173,12.282322,12.207979,16.704589,14.701841,17.319606,44389.0
7,Angeles Crest,-117.909387,34.290855,72841,NaN,12.401353,19.954904,13.303269,15.332582,16.121759,2.931229,887.0
8,Arcadia,-118.037229,34.134077,75808,22.244256,6.236469,10.546328,13.178816,15.400149,14.381356,18.012626,54967.0
9,Arleta,-118.432099,34.241436,65649,22.267747,11.444960,17.824517,12.589153,12.992807,11.611885,11.268931,32949.0


#### Los Angeles Race and Ethnicity Data 

Finally, we will include race and ethnicity data for the Los Angeles neighborhoods. 

In [15]:
#Get the data
url = 'https://usc.data.socrata.com/resource/jxw5-xxv5.json?year=2016&$limit=20000'
response = requests.get(url)
data = response.json()
re_data_all = pd.DataFrame.from_records(data)

print(re_data_all.shape)
re_data_all.head()

(18752, 13)


,count,dataset,date,geoid,location,neighborhood,percent,policy_area,row_id,tract,tract_number,variable,year
0,66,Race & Ethnicity,2016-01-01T00:00:00.000,1400000US06037271500,"(34.01663, -118.4375635)",Mar Vista,2.0899303,Demography,Black_Population_2016_1400000US06037271500,"Census Tract 2715, Los Angeles County, California",271500,Black Population,2016
1,19,Race & Ethnicity,2016-01-01T00:00:00.000,1400000US06037311200,"(34.1714255, -118.3527755)",Burbank,0.5856967,Demography,Black_Population_2016_1400000US06037311200,"Census Tract 3112, Los Angeles County, California",311200,Black Population,2016
2,136,Race & Ethnicity,2016-01-01T00:00:00.000,1400000US06037311300,"(34.173525, -118.342414)",Burbank,3.4482758,Demography,Black_Population_2016_1400000US06037311300,"Census Tract 3113, Los Angeles County, California",311300,Black Population,2016
3,51,Race & Ethnicity,2016-01-01T00:00:00.000,1400000US06037311400,"(34.162038, -118.34958)",Burbank,2.2222223,Demography,Black_Population_2016_1400000US06037311400,"Census Tract 3114, Los Angeles County, California",311400,Black Population,2016
4,161,Race & Ethnicity,2016-01-01T00:00:00.000,1400000US06037311500,"(34.164754, -118.33837)",Burbank,2.9315367,Demography,Black_Population_2016_1400000US06037311500,"Census Tract 3115, Los Angeles County, California",311500,Black Population,2016


In [16]:
#Drop all the NaN values 
re_data_all.dropna(inplace=True)

#Choose only the columns we want
re_data = re_data_all[['neighborhood', 'variable', 'count']]
re_data.sort_values(by = 'neighborhood', inplace = True)

#Cast as int
re_data = re_data.astype({"count": int})

#Sum up the total for each age group by neighborhood (neighborhoods potentially have mutliple tracts; we combine these to get a total for each neighborhood)
re_data = re_data.groupby(['neighborhood', 'variable']).agg({'count': 'sum'})
re_data.reset_index(drop = False, inplace = True)

#Rename columns
re_data.rename(columns={'neighborhood':'Neighborhood', 'variable': 'Race/Ethnicity', 'count': 'Count'}, inplace=True)
re_data.head(15)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Race/Ethnicity,Count
0,Acton,American Indian/Native Population,23
1,Acton,Asian Population,114
2,Acton,Black Population,180
3,Acton,Hispanic Population,1417
4,Acton,Native Hawaiian/Other Pacific Islander Population,5
5,Acton,Other Race Population,56
6,Acton,Population of Two or More Races,271
7,Acton,White Population,5705
8,Adams-Normandie,American Indian/Native Population,0
9,Adams-Normandie,Asian Population,1363


We create a DataFrame with each row being a Neighborhood, and each column being a category. Finally, we merge it with the rest of the LA demographic data in la_data. In this step, we also drop the geographical coordinates in the la_data table so that it contains strictly demographic information. 

In [17]:
counts_list = re_data.groupby('Neighborhood')
result = counts_list['Count'].unique()
counts = pd.DataFrame(result)
counts[['American Indian/Native Population','Asian Population', 'Black Population','Hispanic Population','Native Hawaiian/Other Pacific Islander Population', 'Other Race Population', 'Population of Two or More Races', 'White Population']] = pd.DataFrame(counts.Count.tolist(), index= counts.index)
counts.drop(columns = ['Count'], inplace = True)
counts

la_data = la_data.merge(counts, how = 'left', left_on = 'Neighborhood', right_on = 'Neighborhood')
la_data.iloc[:, 12:] = la_data.iloc[:, 12:].div(la_data['Total Population'], axis=0) * 100

#Don't really need the coordinates, so we can discard those for now 
la_data.drop(columns = ['Latitude', 'Longitude'], inplace = True)

la_data.head(10)

,Neighborhood,Median Income,Under Age 18,Ages 18-24,Ages 25-34,Ages 35-44,Ages 45-54,Ages 55-64,Ages 65 & Older,Total Population,American Indian/Native Population,Asian Population,Black Population,Hispanic Population,Native Hawaiian/Other Pacific Islander Population,Other Race Population,Population of Two or More Races,White Population
0,Acton,83983,17.535794,8.340996,9.667674,10.271903,17.588336,20.596348,15.998949,7613.0,0.302115,1.497439,2.364377,18.612899,0.065677,0.735584,3.559701,74.937607
1,Adams-Normandie,29606,20.946428,18.297966,16.027855,12.304655,11.443768,11.860503,9.118824,18237.0,0.000000,7.473817,20.738060,60.514339,0.060317,0.515436,0.564786,6.267478
2,Agoura Hills,117608,22.520192,6.799345,9.771420,13.350578,17.763818,15.382991,14.411656,18943.0,0.052790,6.181703,1.567861,13.086628,0.000000,0.475110,2.549755,75.637439
3,Agua Dulce,106078,17.722140,8.492882,12.714777,7.756505,15.635739,20.250368,17.427590,4074.0,0.000000,1.669121,0.147275,15.807560,6.136475,67.525773,NaN,NaN
4,Alhambra,53224,17.292348,8.549674,16.339115,13.664180,14.331443,12.968673,16.854567,84974.0,0.304799,50.363641,1.758185,36.821851,0.124744,0.055311,1.356886,9.087486
5,Alondra Park,57177,18.681319,9.442409,11.294261,10.826211,13.492063,15.934066,20.329670,4914.0,0.000000,33.455433,4.192104,30.301180,0.793651,1.668702,30.769231,NaN
6,Altadena,82676,19.430490,7.353173,12.282322,12.207979,16.704589,14.701841,17.319606,44389.0,0.083354,7.465814,21.410710,26.517831,0.000000,0.653315,4.911127,38.502782
7,Angeles Crest,72841,NaN,12.401353,19.954904,13.303269,15.332582,16.121759,2.931229,887.0,0.000000,1.014656,9.357384,23.111612,5.186020,62.683202,NaN,NaN
8,Arcadia,75808,22.244256,6.236469,10.546328,13.178816,15.400149,14.381356,18.012626,54967.0,0.272891,58.402314,1.635527,12.884094,0.136446,0.249240,2.748922,22.866447
9,Arleta,65649,22.267747,11.444960,17.824517,12.589153,12.992807,11.611885,11.268931,32949.0,0.206380,8.828796,1.025828,76.405961,0.000000,0.176030,0.667699,6.370451


This concludes the preparation of the LA demographic data. 

#### Irvine Data 

Next, we examine the same metrics for the city of Irvine in Orange County, where the client's original restaurant is located.


In [18]:
# The code was removed by Watson Studio for sharing.

In [19]:
#Read the Irvine data (uploaded as a csv to IBM Cloud Object Storage)
irvine_data = pd.read_csv(body, header = 1)
irvine_data.head()

,id,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Percent Estimate!!SEX AND AGE!!Total population,Percent Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population!!Male,Percent Estimate!!SEX AND AGE!!Total population!!Male,Percent Margin of Error!!SEX AND AGE!!Total population!!Male,...,"Percent Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female"
0,1600000US0636770,"Irvine city, California",282584,66,282584,(X),138076,4009,48.9,1.4,...,168948,(X),82299,4228,48.7,1.6,86649,4804,51.3,1.6


The following code filters out and formats the data for race/ethnicity to be used in the analysis:

In [20]:
irvine_eth = irvine_data.filter(regex='Percent Estimate',axis=1)
irvine_eth = irvine_eth.filter(regex='HISPANIC OR LATINO AND RACE',axis=1)

#Fix the column labels
column_labels = ['Total Population']
for name in irvine_eth.columns[1:]: 
    new_name = name.replace("Percent Estimate!!HISPANIC OR LATINO AND RACE!!Total population!!", "")
    column_labels.append(new_name)    
irvine_eth.columns = column_labels

#Drop unnecessary columns 
irvine_eth.drop(irvine_eth.iloc[:, 2:6], inplace = True, axis = 1)
irvine_eth.drop(irvine_eth.iloc[:, -2:], inplace = True, axis = 1)
irvine_eth.drop(columns = ['Not Hispanic or Latino', 'Total Population'], inplace = True)

#Format to match LA data
column_labels = ['Hispanic Population', 'White Population']
for name in irvine_eth.columns[2:]: 
    new_name = name.replace("Not Hispanic or Latino!!", "")
    column_labels.append(new_name)    
irvine_eth.columns = column_labels

irvine_eth = irvine_eth.sort_index(axis=1)
irvine_eth.columns = ['American Indian/Native Population', 'Asian Population', 'Black Population', 'Hispanic Population', 'Native Hawaiian/Other Pacific Islander Population', 'Other Race Population', 'Population of Two or More Races', 'White Population']

irvine_eth.head()

,American Indian/Native Population,Asian Population,Black Population,Hispanic Population,Native Hawaiian/Other Pacific Islander Population,Other Race Population,Population of Two or More Races,White Population
0,0.4,42.5,1.0,12.5,0.1,0.4,4.2,38.9


And the following code filters out and formats the data for age distribution to be used in the analysis:

In [21]:
irvine_age_all = irvine_data.filter(like='SEX AND AGE',axis=1)
irvine_age_all = irvine_age_all.filter(regex='Percent Estimate',axis=1)

irvine_age_all.drop(list(irvine_age_all.filter(regex='Error')), axis = 1, inplace = True)

irvine_age_all = irvine_age_all.iloc[:, 0:18]
irvine_age_all

,Percent Estimate!!SEX AND AGE!!Total population,Percent Estimate!!SEX AND AGE!!Total population!!Male,Percent Estimate!!SEX AND AGE!!Total population!!Female,Percent Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),Percent Estimate!!SEX AND AGE!!Total population!!Under 5 years,Percent Estimate!!SEX AND AGE!!Total population!!5 to 9 years,Percent Estimate!!SEX AND AGE!!Total population!!10 to 14 years,Percent Estimate!!SEX AND AGE!!Total population!!15 to 19 years,Percent Estimate!!SEX AND AGE!!Total population!!20 to 24 years,Percent Estimate!!SEX AND AGE!!Total population!!25 to 34 years,Percent Estimate!!SEX AND AGE!!Total population!!35 to 44 years,Percent Estimate!!SEX AND AGE!!Total population!!45 to 54 years,Percent Estimate!!SEX AND AGE!!Total population!!55 to 59 years,Percent Estimate!!SEX AND AGE!!Total population!!60 to 64 years,Percent Estimate!!SEX AND AGE!!Total population!!65 to 74 years,Percent Estimate!!SEX AND AGE!!Total population!!75 to 84 years,Percent Estimate!!SEX AND AGE!!Total population!!85 years and over,Percent Estimate!!SEX AND AGE!!Total population!!Median age (years)
0,282584,48.9,51.1,(X),6.4,6.0,6.5,7.7,9.4,16.2,14.7,12.6,5.6,4.4,6.3,3.1,1.2,(X)


In [22]:
#Fix the column labels
column_labels = ['Total Population']
for name in irvine_age_all.columns[1:]: 
    new_name = name.replace("Percent Estimate!!SEX AND AGE!!Total population!!", "")
    column_labels.append(new_name)
irvine_age_all.columns = column_labels

#Drop unnecessary columns 
irvine_age = irvine_age_all.drop(columns = ['Male', 'Female','Sex ratio (males per 100 females)', 'Median age (years)'], axis = 1)

#Create same age categories as the LA neighborhoods
irvine_age['Under Age 18'] = irvine_age.iloc[:,1:5].sum(axis=1)
#(these ages aren't exact but close enough lol)
irvine_age['Ages 18-24'] = irvine_age['20 to 24 years']
irvine_age['Ages 25-34'] = irvine_age['25 to 34 years']
irvine_age['Ages 35-44'] = irvine_age['35 to 44 years']
irvine_age['Ages 45-54'] = irvine_age['45 to 54 years']
irvine_age['Ages 55-64']= irvine_age['55 to 59 years'] + irvine_age['60 to 64 years']
irvine_age['Ages 65 & Older'] = irvine_age['65 to 74 years'] + irvine_age['75 to 84 years'] + irvine_age['85 years and over'] 

#Move Total Population column to the end
total_pop = irvine_age.pop('Total Population')
irvine_age['Total Population'] = total_pop

#Drop all the original categories 
irvine_age.drop(irvine_age.iloc[:, 0:13], inplace = True, axis = 1) 

irvine_age.astype(float, copy = False)
irvine_age.head()

,Under Age 18,Ages 18-24,Ages 25-34,Ages 35-44,Ages 45-54,Ages 55-64,Ages 65 & Older,Total Population
0,26.6,9.4,16.2,14.7,12.6,10.0,10.6,282584


We obtain Irvine's median income from the census information, and create a DataFrame to concatenate with the age and race/ethnicity data. 

In [23]:
data = {'Neighborhood': ['Irvine'], 'Median Income': [100969]}
temp_df = pd.DataFrame(data)
irvine = pd.concat([temp_df, irvine_age, irvine_eth], axis = 1)
irvine.head()

,Neighborhood,Median Income,Under Age 18,Ages 18-24,Ages 25-34,Ages 35-44,Ages 45-54,Ages 55-64,Ages 65 & Older,Total Population,American Indian/Native Population,Asian Population,Black Population,Hispanic Population,Native Hawaiian/Other Pacific Islander Population,Other Race Population,Population of Two or More Races,White Population
0,Irvine,100969,26.6,9.4,16.2,14.7,12.6,10.0,10.6,282584,0.4,42.5,1.0,12.5,0.1,0.4,4.2,38.9


We then append this to the existing LA neighborhoods DataFrame so that we can cluster the data.

In [24]:
all_demo = pd.concat([irvine, la_data], sort = False)
all_demo.reset_index(inplace = True, drop = True)
all_demo.head()

,Neighborhood,Median Income,Under Age 18,Ages 18-24,Ages 25-34,Ages 35-44,Ages 45-54,Ages 55-64,Ages 65 & Older,Total Population,American Indian/Native Population,Asian Population,Black Population,Hispanic Population,Native Hawaiian/Other Pacific Islander Population,Other Race Population,Population of Two or More Races,White Population
0,Irvine,100969,26.600000,9.400000,16.200000,14.700000,12.600000,10.000000,10.600000,282584.0,0.400000,42.500000,1.000000,12.500000,0.100000,0.400000,4.200000,38.900000
1,Acton,83983,17.535794,8.340996,9.667674,10.271903,17.588336,20.596348,15.998949,7613.0,0.302115,1.497439,2.364377,18.612899,0.065677,0.735584,3.559701,74.937607
2,Adams-Normandie,29606,20.946428,18.297966,16.027855,12.304655,11.443768,11.860503,9.118824,18237.0,0.000000,7.473817,20.738060,60.514339,0.060317,0.515436,0.564786,6.267478
3,Agoura Hills,117608,22.520192,6.799345,9.771420,13.350578,17.763818,15.382991,14.411656,18943.0,0.052790,6.181703,1.567861,13.086628,0.000000,0.475110,2.549755,75.637439
4,Agua Dulce,106078,17.722140,8.492882,12.714777,7.756505,15.635739,20.250368,17.427590,4074.0,0.000000,1.669121,0.147275,15.807560,6.136475,67.525773,NaN,NaN


### Clustering <a name="clusterLA"></a>   

We now run K-means clustering on this demographic data. This gives us candidates for neighborhoods that are similar in demographic composition to Irvine. 

In [25]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#Drop the 'Neighborhood' column because it is a categorical variable
df = all_demo.drop(columns = ['Neighborhood'], axis = 1)

#Normalize the data using StandardScaler()
X = df.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)

#Run K-Means Clustering 
num_clusters = 15 #create 15 clusters of neighborhoods 

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init= 15)
k_means.fit(cluster_dataset)
labels = k_means.labels_

#Add the cluster labels back to our original dataframe! 
all_demo['Cluster Label'] = labels
all_demo.head()

,Neighborhood,Median Income,Under Age 18,Ages 18-24,Ages 25-34,Ages 35-44,Ages 45-54,Ages 55-64,Ages 65 & Older,Total Population,American Indian/Native Population,Asian Population,Black Population,Hispanic Population,Native Hawaiian/Other Pacific Islander Population,Other Race Population,Population of Two or More Races,White Population,Cluster Label
0,Irvine,100969,26.600000,9.400000,16.200000,14.700000,12.600000,10.000000,10.600000,282584.0,0.400000,42.500000,1.000000,12.500000,0.100000,0.400000,4.200000,38.900000,0
1,Acton,83983,17.535794,8.340996,9.667674,10.271903,17.588336,20.596348,15.998949,7613.0,0.302115,1.497439,2.364377,18.612899,0.065677,0.735584,3.559701,74.937607,2
2,Adams-Normandie,29606,20.946428,18.297966,16.027855,12.304655,11.443768,11.860503,9.118824,18237.0,0.000000,7.473817,20.738060,60.514339,0.060317,0.515436,0.564786,6.267478,14
3,Agoura Hills,117608,22.520192,6.799345,9.771420,13.350578,17.763818,15.382991,14.411656,18943.0,0.052790,6.181703,1.567861,13.086628,0.000000,0.475110,2.549755,75.637439,2
4,Agua Dulce,106078,17.722140,8.492882,12.714777,7.756505,15.635739,20.250368,17.427590,4074.0,0.000000,1.669121,0.147275,15.807560,6.136475,67.525773,NaN,NaN,12


We can check to see which cluster Irvine is in, and which neighborhoods are also in the same cluster (and are therefore similar).

In [26]:
irvine_label = all_demo.loc[all_demo['Neighborhood']=='Irvine', 'Cluster Label']
print("Irvine is in Cluster: ", irvine_label.iloc[0])

similar = pd.DataFrame(all_demo.loc[all_demo['Cluster Label'] == irvine_label.iloc[0]])
similar

Irvine is in Cluster:  0


,Neighborhood,Median Income,Under Age 18,Ages 18-24,Ages 25-34,Ages 35-44,Ages 45-54,Ages 55-64,Ages 65 & Older,Total Population,American Indian/Native Population,Asian Population,Black Population,Hispanic Population,Native Hawaiian/Other Pacific Islander Population,Other Race Population,Population of Two or More Races,White Population,Cluster Label
0,Irvine,100969,26.600000,9.400000,16.200000,14.700000,12.600000,10.000000,10.600000,282584.0,0.400000,42.500000,1.000000,12.500000,0.100000,0.400000,4.200000,38.900000,0
83,Glendale,57112,17.568822,7.550456,15.924880,13.057713,14.860360,13.910626,17.127144,200372.0,0.176172,15.523127,1.316551,17.454535,0.049408,0.135748,2.768850,58.382908,0
121,Lancaster,56069,28.423251,9.291475,14.940394,12.792005,12.812570,11.583175,10.157129,155605.0,0.155522,3.427268,19.977507,36.653064,0.245493,0.139456,2.623309,33.314482,0
132,Long Beach,50985,22.849530,10.639079,17.105034,13.694558,13.542326,11.154377,11.015096,470990.0,0.331854,13.088601,12.627232,42.704091,0.881335,0.190662,3.165885,28.206331,0
162,Palmdale,63317,30.270144,10.644994,13.419727,12.102884,13.251580,11.323741,8.986930,162358.0,0.354772,4.327474,12.894961,61.820175,0.043115,0.345533,2.214243,21.529583,0
167,Pasadena,62825,18.944214,8.162153,17.684200,14.735320,13.147032,11.772471,15.554609,143173.0,0.062163,14.817738,9.747648,33.521684,0.152962,0.363197,2.542379,36.714325,0
173,Pomona,54242,25.681998,14.576994,14.573723,13.261747,12.235724,9.815276,9.854538,152823.0,0.297730,9.068661,6.385819,70.033307,0.342880,0.287260,1.584840,11.691957,0
193,Santa Clarita,88987,25.955314,9.136960,12.617181,13.638179,15.241403,11.985560,11.425404,190304.0,0.136624,10.835295,3.926349,34.241004,0.077245,0.163948,3.809694,49.284303,0
220,Torrance,76866,20.855648,7.380185,12.826521,13.161921,14.714602,14.333434,16.727690,146392.0,0.231570,34.094759,2.457785,17.736625,0.530767,0.334035,4.670337,40.569157,0


Thus, based on the age distribution, median income, and race/ethnicity data, it seems that the neighborhoods most similar to Irvine are: **Glendale, Lancaster, Long Beach, Palmdale, Pasadena, Pomona, Santa Clarita**, and **Torrance**. These are potential candidates for the client's new restaurant location. To further refine our recommendation, we next turn to using Foursquare data. 

## Part 2: Foursquare Data  <a name="foursquare"></a>

In this next section, we use the Foursquare API to gather information about the business landscape of various neighborhoods. 

### Preparing the Data <a name="datafoursquare"></a>   

First, we will get the most common types of venues for each neighborhood. We return to our 'neighborhoods' DataFrame, which contains a list of the LA neighborhoods as well as their geographical coordinates. 

In [27]:
neighborhoods.head()

,Neighborhood,Longitude,Latitude
0,Acton,-118.185799,34.495516
1,Adams-Normandie,-118.300288,34.031411
2,Agoura Hills,-118.760944,34.150734
3,Agua Dulce,-118.313371,34.508909
4,Alhambra,-118.135494,34.083967


In [28]:
#@hidden 

#Define Foursquare credentials and version
CLIENT_ID = 'GVWEAF241OAMHXLTB4HIHSQEOFOW5FYIGFMVI30YZH4GVYXA' #Foursquare ID
CLIENT_SECRET = 'UYJDYFU4G2TSXSLAXXSTYPGXJMERLLYHTEORL5VNQOGB5T5Y' #Foursquare Secret
VERSION = '20180605' # Foursquare API version

We would like to explore venues in both Irvine and LA using the Foursquare API. We first add Irvine to our 'neighborhoods' DataFrame.

In [29]:
#Add Irvine to the neighborhoods DataFrame 
irvine_lat = 33.6846
irvine_long = -117.8265

data = {'Neighborhood': ['Irvine'], 'Longitude': [irvine_long], 'Latitude': [irvine_lat]}
df = pd.DataFrame(data)
all_neighborhoods = pd.concat([df, neighborhoods], sort = False)
all_neighborhoods.reset_index(inplace = True, drop = True)
all_neighborhoods.sort_values(by = 'Neighborhood', inplace = True)
all_neighborhoods.head()

,Neighborhood,Longitude,Latitude
1,Acton,-118.185799,34.495516
2,Adams-Normandie,-118.300288,34.031411
3,Agoura Hills,-118.760944,34.150734
4,Agua Dulce,-118.313371,34.508909
5,Alhambra,-118.135494,34.083967


Then, we define the following function to get nearby venues in each neighborhood, make the API call, clean the json, and structure it into a DataFrame.

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 3000 # define radius = 3000m 

#function to get nearby venues in each neighborhood
def getNearbyVenues(names, latitudes, longitudes, radius = 3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print("Processing: {}".format(name))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
#Contains all the venues for each neighborhood
venues = getNearbyVenues(names=all_neighborhoods['Neighborhood'],
                                   latitudes=all_neighborhoods['Latitude'],
                                   longitudes=all_neighborhoods['Longitude']
                                  )

venues.head()

Processing: Acton
Processing: Adams-Normandie
Processing: Agoura Hills
Processing: Agua Dulce
Processing: Alhambra
Processing: Alondra Park
Processing: Altadena
Processing: Angeles Crest
Processing: Arcadia
Processing: Arleta
Processing: Arlington Heights
Processing: Artesia
Processing: Athens
Processing: Atwater Village
Processing: Avalon
Processing: Avocado Heights
Processing: Azusa
Processing: Baldwin Hills/Crenshaw
Processing: Baldwin Park
Processing: Bel-Air
Processing: Bell
Processing: Bell Gardens
Processing: Bellflower
Processing: Beverly Crest
Processing: Beverly Grove
Processing: Beverly Hills
Processing: Beverlywood
Processing: Boyle Heights
Processing: Bradbury
Processing: Brentwood
Processing: Broadway-Manchester
Processing: Burbank
Processing: Calabasas
Processing: Canoga Park
Processing: Carson
Processing: Carthay
Processing: Castaic
Processing: Castaic Canyons
Processing: Central-Alameda
Processing: Century City
Processing: Cerritos
Processing: Charter Oak
Processing: C

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,34.495516,-118.185799,La Cabaña,34.479308,-118.166997,Mexican Restaurant
1,Acton,34.495516,-118.185799,SUBWAY,34.493552,-118.188909,Sandwich Place
2,Acton,34.495516,-118.185799,Crazy Otto's Diner,34.490733,-118.162548,Breakfast Spot
3,Acton,34.495516,-118.185799,Jack in the Box,34.492398,-118.199205,Fast Food Restaurant
4,Acton,34.495516,-118.185799,SUBWAY,34.492420,-118.198858,Sandwich Place


To check the size of the resulting dataframe: 

In [32]:
print(venues.shape)
venues.head()

(22335, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,34.495516,-118.185799,La Cabaña,34.479308,-118.166997,Mexican Restaurant
1,Acton,34.495516,-118.185799,SUBWAY,34.493552,-118.188909,Sandwich Place
2,Acton,34.495516,-118.185799,Crazy Otto's Diner,34.490733,-118.162548,Breakfast Spot
3,Acton,34.495516,-118.185799,Jack in the Box,34.492398,-118.199205,Fast Food Restaurant
4,Acton,34.495516,-118.185799,SUBWAY,34.492420,-118.198858,Sandwich Place


To check how many venues were returned for each neighborhood: 

In [33]:
venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

# Count the number of locations per Venue Category
venues.groupby('Venue Category').count()['Neighborhood'].sort_values(ascending=False).head(10)

There are 432 uniques categories.


Venue Category
Mexican Restaurant      1221
Coffee Shop              975
Fast Food Restaurant     878
Pizza Place              704
Sandwich Place           657
Grocery Store            655
Burger Joint             599
Park                     547
American Restaurant      510
Convenience Store        462
Name: Neighborhood, dtype: int64

### Clustering the Data <a name="clusterfoursquare"></a>   

We next want to analyze each neighborhood and prepare it for K-means clustering. We first perform onehot encoding. 

In [34]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#IMPORTANT: apparently there is a venue type called "Neighborhood" which is why we must now call our column "NEIGHBORHOOD NAME"
onehot['Neighborhood Name'] = venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape) 
onehot.head()

(22335, 433)


,Neighborhood Name,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, we group rows by neighborhood and take the mean of the frequency of occurrence of each category.

In [35]:
grouped = onehot.groupby('Neighborhood Name').mean().reset_index()
grouped.head()

,Neighborhood Name,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Acton,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.041667,0.00,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,Adams-Normandie,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.020000,0.01,...,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,Agoura Hills,0.011905,0.0,0.0,0.0,0.000000,0.0,0.0,0.011905,0.00,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Agua Dulce,0.000000,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.071429,0.0,0.0,0.0,0.0,0.0
4,Alhambra,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.020000,0.00,...,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


Next, we put the information in a DataFrame, using the following function (from the lab) that sorts the venues in descending order. 

In [36]:
#function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories = row_categories[row_categories != 0]   #in case neighborhoods don't have that many top venues 
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood Name'] = grouped['Neighborhood Name']

for ind in np.arange(grouped.shape[0]):
    length = len(return_most_common_venues(grouped.iloc[ind, :], num_top_venues))  #Will fill the rest of the row with NaN if a neighborhood does not have enough data for 10 venues
    neighborhoods_venues_sorted.iloc[ind, 1:length+1] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Park,Café,Mexican Restaurant,Construction & Landscaping,Sandwich Place,Fast Food Restaurant,Sushi Restaurant,Convenience Store,Arts & Entertainment,Breakfast Spot
1,Adams-Normandie,Mexican Restaurant,Korean Restaurant,Food Truck,Pizza Place,Café,Science Museum,Taco Place,Coffee Shop,Burger Joint,Sandwich Place
2,Agoura Hills,Deli / Bodega,Park,Gym,Hotel,Italian Restaurant,Mexican Restaurant,Trail,Pharmacy,Fast Food Restaurant,Thai Restaurant
3,Agua Dulce,Winery,Trail,Pizza Place,Park,Mexican Restaurant,Home Service,Grocery Store,Gift Shop,Electronics Store,Convenience Store
4,Alhambra,Chinese Restaurant,Park,Mexican Restaurant,Vietnamese Restaurant,Convenience Store,Burger Joint,Café,Grocery Store,Fast Food Restaurant,Italian Restaurant


We will again use k-means clustering to cluster the neighborhoods. The goal of this part is to find a neighborhood with a similar business/competitor landscape as the original Irvine location. 

In [38]:
# set number of clusters
num_clusters = 20

grouped_clustering = grouped.drop('Neighborhood Name', 1)

# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=num_clusters, n_init = 15).fit(grouped_clustering)

labels = kmeans.labels_
neighborhoods_venues_sorted["Cluster Label"] = labels 

neighborhoods_venues_sorted.head()
venues_merged = all_neighborhoods.merge(neighborhoods_venues_sorted, how ='inner', left_on = 'Neighborhood', right_on = "Neighborhood Name")
venues_merged.drop('Neighborhood Name', 1, inplace = True)

venues_merged.head()

,Neighborhood,Longitude,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
0,Acton,-118.185799,34.495516,Park,Café,Mexican Restaurant,Construction & Landscaping,Sandwich Place,Fast Food Restaurant,Sushi Restaurant,Convenience Store,Arts & Entertainment,Breakfast Spot,19
1,Adams-Normandie,-118.300288,34.031411,Mexican Restaurant,Korean Restaurant,Food Truck,Pizza Place,Café,Science Museum,Taco Place,Coffee Shop,Burger Joint,Sandwich Place,4
2,Agoura Hills,-118.760944,34.150734,Deli / Bodega,Park,Gym,Hotel,Italian Restaurant,Mexican Restaurant,Trail,Pharmacy,Fast Food Restaurant,Thai Restaurant,18
3,Agua Dulce,-118.313371,34.508909,Winery,Trail,Pizza Place,Park,Mexican Restaurant,Home Service,Grocery Store,Gift Shop,Electronics Store,Convenience Store,19
4,Alhambra,-118.135494,34.083967,Chinese Restaurant,Park,Mexican Restaurant,Vietnamese Restaurant,Convenience Store,Burger Joint,Café,Grocery Store,Fast Food Restaurant,Italian Restaurant,13


We can now examine which cluster Irvine is in, and which neighborhoods are also in that cluter. 

In [39]:
irvine_label = venues_merged.loc[venues_merged['Neighborhood']=='Irvine', 'Cluster Label']
print("Irvine is in cluster: ", irvine_label.iloc[0])

similar = pd.DataFrame(venues_merged.loc[venues_merged['Cluster Label'] == irvine_label.iloc[0]])
print("There are {} other venues in cluster {}".format(similar.shape[0]-1, irvine_label.iloc[0]))
similar

Irvine is in cluster:  18
There are 84 other venues in cluster 18


,Neighborhood,Longitude,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
2,Agoura Hills,-118.760944,34.150734,Deli / Bodega,Park,Gym,Hotel,Italian Restaurant,Mexican Restaurant,Trail,Pharmacy,Fast Food Restaurant,Thai Restaurant,18
5,Alondra Park,-118.334949,33.888520,Japanese Restaurant,American Restaurant,Mediterranean Restaurant,Grocery Store,Vietnamese Restaurant,Coffee Shop,Burger Joint,Sushi Restaurant,Fast Food Restaurant,Italian Restaurant,18
10,Artesia,-118.080648,33.867590,Indian Restaurant,Bakery,Café,Coffee Shop,Grocery Store,Sandwich Place,Park,BBQ Joint,Burger Joint,Dessert Shop,18
12,Atwater Village,-118.265063,34.126138,Bakery,Coffee Shop,Café,Mediterranean Restaurant,Ice Cream Shop,Middle Eastern Restaurant,Mexican Restaurant,Brewery,Burger Joint,Taco Place,18
13,Avalon,-118.331202,33.333827,Hotel,Seafood Restaurant,Pizza Place,Bar,Trail,Beach,Harbor / Marina,History Museum,Ice Cream Shop,Mexican Restaurant,18
16,Baldwin Hills/Crenshaw,-118.354633,34.015489,Mexican Restaurant,Park,Coffee Shop,Art Gallery,Fried Chicken Joint,Seafood Restaurant,Cosmetics Shop,New American Restaurant,Convenience Store,Ice Cream Shop,18
23,Beverly Grove,-118.371392,34.075648,Coffee Shop,Café,American Restaurant,Mexican Restaurant,Salad Place,Clothing Store,Cocktail Bar,French Restaurant,Grocery Store,Hotel,18
24,Beverly Hills,-118.402109,34.078539,Hotel,American Restaurant,Boutique,Clothing Store,Italian Restaurant,Coffee Shop,Park,Ice Cream Shop,Café,Men's Store,18
25,Beverlywood,-118.393787,34.044646,Coffee Shop,American Restaurant,Sushi Restaurant,Park,Liquor Store,Grocery Store,New American Restaurant,Café,Art Gallery,Farmers Market,18
30,Burbank,-118.323533,34.187860,Burger Joint,Mexican Restaurant,American Restaurant,Park,Coffee Shop,Sandwich Place,Donut Shop,Hotel,Grocery Store,Pizza Place,18


Though this gives us valuable information about the top types of venues for each neighborhood, this clustering procedure based just on the Foursquare data was not particularly informative, as the number of neighborhoods in Irvine's cluster is quite large. Therefore, we now include the demographic data from earlier, and perform k-means clustering on the entire dataset.  

In [40]:
#all_demo is the table containing demographic data for both LA and Irvine
all_demo.sort_values(by = ['Neighborhood'], inplace = True)

#Drop categorical values
df = all_demo.drop(columns = ['Neighborhood', 'Cluster Label'], axis = 1)

# Normalize the data using StandardScaler()
X = df.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
dem_cluster = pd.DataFrame(cluster_dataset)
dem_cluster['Neighborhood Name'] = all_demo['Neighborhood'].to_list()

In [46]:
#Create a master table (all_clustering) containing all of the demographic and the Foursquare data 
all_clustering = grouped.merge(dem_cluster, how = 'inner', left_on = 'Neighborhood Name', right_on = 'Neighborhood Name')
clustered_neighborhoods = all_clustering['Neighborhood Name']
all_clustering.drop('Neighborhood Name', axis = 1, inplace = True)
all_clustering.head()

#Run K-Means Clustering 
num_clusters = 15

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init= 15)
k_means.fit(all_clustering)
labels = k_means.labels_

#Create a table of all neighborhoods with cluster labels
final = all_neighborhoods[all_neighborhoods['Neighborhood'].isin(clustered_neighborhoods)]
final.sort_values(by = ['Neighborhood'], inplace = True)
final['Cluster Label'] = labels
final.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,Longitude,Latitude,Cluster Label
1,Acton,-118.185799,34.495516,4
2,Adams-Normandie,-118.300288,34.031411,7
3,Agoura Hills,-118.760944,34.150734,4
4,Agua Dulce,-118.313371,34.508909,6
5,Alhambra,-118.135494,34.083967,12


To better visualize the resulting neighborhood clusters, we create a folium map. Neighborhoods in the same cluster are marked with the same color. 

In [47]:
# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final['Latitude'], final['Longitude'], final['Neighborhood'], final['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Once again, we identify the other neighborhoods in the same cluster as Irvine. 

In [48]:
irvine_label = final.loc[final['Neighborhood']=='Irvine', 'Cluster Label']
print("Irvine is in cluster:", irvine_label.iloc[0])

final_similar = pd.DataFrame(final.loc[final['Cluster Label'] == irvine_label.iloc[0]])
print("There are {} other venues in cluster {}".format(final_similar.shape[0]-1, irvine_label.iloc[0]))
final_similar

Irvine is in cluster: 1
There are 8 other venues in cluster 1


,Neighborhood,Longitude,Latitude,Cluster Label
84,Glendale,-118.246813,34.181902,1
0,Irvine,-117.826500,33.684600,1
125,Lancaster,-118.174337,34.692750,1
136,Long Beach,-118.160709,33.805217,1
166,Palmdale,-118.086611,34.598026,1
171,Pasadena,-118.135127,34.161660,1
177,Pomona,-117.762309,34.058683,1
197,Santa Clarita,-118.499399,34.410701,1
226,Torrance,-118.341684,33.834646,1


As it happens, our list of neighborhoods in the same cluster as Irvine is the same as when we performed clustering with just the demographic data. We now examine the number of Chinese restaurants in these neighborhoods, as per the client's desire to open in a neighborhood with fewer Chinese restaurants. 

In [49]:
potential_neighborhoods = final_similar.Neighborhood.to_list()
potential_neighborhoods.remove("Irvine")

chinese_restaurants = onehot.groupby(['Neighborhood Name']).sum().reset_index()
chinese_restaurants = chinese_restaurants[['Neighborhood Name','Chinese Restaurant']]
chinese_restaurants.rename(columns={'Chinese Restaurant':'Num_Chinese Restaurants'}, inplace=True)
chinese_restaurants.head(10)

num_chinese = chinese_restaurants[chinese_restaurants["Neighborhood Name"].isin(potential_neighborhoods)]
num_chinese.sort_values(by = "Num_Chinese Restaurants", ascending = False, inplace = True)
num_chinese

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood Name,Num_Chinese Restaurants
176,Pomona,4
126,Lancaster,2
165,Palmdale,2
195,Santa Clarita,2
82,Glendale,1
170,Pasadena,1
223,Torrance,1
135,Long Beach,0


Based on these results, the recommended neighborhoods in order of least to most Chinese restaurants are: **Long Beach**, **Torrance**, **Pasadena**, **Glendale**, **Santa Clarita**, **Palmdale**, **Lancaster**, and **Pomona**. 

## 5. Results <a name="results"></a>

The purpose of this study was to identify potential neighborhoods for our client to open the second location of their fast-casual Chinese restaurant. We aimed to find neighborhoods that would be similar to the client's original restaurant location in Irvine, California. 

To accomplish this, we researched demographic data, including median income, age distribution, and race/ethnicity, and the existing business landscape of Los Angeles neighborhoods via the Foursquare API. After performing K-means clustering, it was determined that the neighborhoods most similar to Irvine are: **Long Beach**, **Torrance**, **Pasadena**, **Glendale**, **Santa Clarita**, **Palmdale**, **Lancaster**, and **Pomona**. These represent potential locations for the client's newest venture. Further details are provided in the business report. 
